# Training

In [1]:
# Prediction margin: the only parameter to set. Recommended: margin in {5, 10, 15, 20} (aka 0.5, 1, 1.5, 2 seconds)
margin = 5

## Import libraries and define utility functions

In [2]:
import pandas as pd
import numpy as np
import sys
import random
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from math import floor
from tensorflow import keras
from tensorflow.keras import layers, callbacks
import tensorflow as tf

In [3]:
# If you have more than one GPU in your system, the GPU with the lowest ID will be selected by default.
# https://www.tensorflow.org/guide/gpu#using_a_single_gpu_on_a_multi-gpu_system
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 2


In [4]:
mean = lambda l: sum(l) / len(l)

In [5]:
def serialize_perf(model_name, seed, columns_name, training_columns, params, params_idx, 
                   history, best_cost, best_thr, all_cost, all_thr, perf):
    f_path = results_path + model_name + "-" + columns_name + "-s" + str(seed) + "-p" + str(params_idx)
    to_serialize = (training_columns, params, history, best_cost, best_thr, all_cost, all_thr, perf)
    with open(f_path, "wb") as file:
        pickle.dump(to_serialize, file)

#### Set up global variables

In [6]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [7]:
results_path = "results_" + str(margin) + "/"
models_path = "models_" + str(margin) + "/"
seeds = [100, 200, 300]

def set_determinism(seed):
    tf.keras.utils.set_random_seed(seed)
    tf.config.experimental.enable_op_determinism()

In [8]:
patience = 100
epochs = 1500
batch_size = 32
learning_rate = 0.0001
validation_split = 0.2

In [9]:
w5_features_no_diff = [
 'Gz_mean_w5',
 'Ax_mean_w5',
 'Ay_mean_w5',
 'Gz_std_w5',
 'Ax_std_w5',
 'Ay_std_w5',
 'Gz_min_w5',
 'Ax_min_w5',
 'Ay_min_w5',
 'Gz_max_w5',
 'Ax_max_w5',
 'Ay_max_w5'
]

w5_features_diff = [
 'differencing_Gz_mean_w5',
 'differencing_Ax_mean_w5',
 'differencing_Ay_mean_w5',
 'differencing_Gz_std_w5',
 'differencing_Ax_std_w5',
 'differencing_Ay_std_w5',
 'differencing_Gz_min_w5',
 'differencing_Ax_min_w5',
 'differencing_Ay_min_w5',
 'differencing_Gz_max_w5',
 'differencing_Ax_max_w5',
 'differencing_Ay_max_w5',
]

w10_features_no_diff = [
 'Gz_mean_w10',
 'Ax_mean_w10',
 'Ay_mean_w10',
 'Gz_std_w10',
 'Ax_std_w10',
 'Ay_std_w10',
 'Gz_min_w10',
 'Ax_min_w10',
 'Ay_min_w10',
 'Gz_max_w10',
 'Ax_max_w10',
 'Ay_max_w10'
]

w10_features_diff = [
 'differencing_Gz_mean_w10',
 'differencing_Ax_mean_w10',
 'differencing_Ay_mean_w10',
 'differencing_Gz_std_w10',
 'differencing_Ax_std_w10',
 'differencing_Ay_std_w10',
 'differencing_Gz_min_w10',
 'differencing_Ax_min_w10',
 'differencing_Ay_min_w10',
 'differencing_Gz_max_w10',
 'differencing_Ax_max_w10',
 'differencing_Ay_max_w10'
]

w15_features_no_diff = [
 'Gz_mean_w15',
 'Ax_mean_w15',
 'Ay_mean_w15',
 'Gz_std_w15',
 'Ax_std_w15',
 'Ay_std_w15',
 'Gz_min_w15',
 'Ax_min_w15',
 'Ay_min_w15',
 'Gz_max_w15',
 'Ax_max_w15',
 'Ay_max_w15'
]

w15_features_diff = [
 'differencing_Gz_mean_w15',
 'differencing_Ax_mean_w15',
 'differencing_Ay_mean_w15',
 'differencing_Gz_std_w15',
 'differencing_Ax_std_w15',
 'differencing_Ay_std_w15',
 'differencing_Gz_min_w15',
 'differencing_Ax_min_w15',
 'differencing_Ay_min_w15',
 'differencing_Gz_max_w15',
 'differencing_Ax_max_w15',
 'differencing_Ay_max_w15'
]

w20_features_no_diff = [
 'Gz_mean_w20',
 'Ax_mean_w20',
 'Ay_mean_w20',
 'Gz_std_w20',
 'Ax_std_w20',
 'Ay_std_w20',
 'Gz_min_w20',
 'Ax_min_w20',
 'Ay_min_w20',
 'Gz_max_w20',
 'Ax_max_w20',
 'Ay_max_w20'
]

w20_features_diff = [
 'differencing_Gz_mean_w20',
 'differencing_Ax_mean_w20',
 'differencing_Ay_mean_w20',
 'differencing_Gz_std_w20',
 'differencing_Ax_std_w20',
 'differencing_Ay_std_w20',
 'differencing_Gz_min_w20',
 'differencing_Ax_min_w20',
 'differencing_Ay_min_w20',
 'differencing_Gz_max_w20',
 'differencing_Ax_max_w20',
 'differencing_Ay_max_w20',
]

features = {
    "all_features": w5_features_no_diff + w10_features_no_diff + w15_features_no_diff + w20_features_no_diff + w5_features_diff + w10_features_diff + w15_features_diff + w20_features_diff + ['label'], 
    "w5_features": w5_features_no_diff + w5_features_diff + ['label'], 
    "w10_features": w10_features_no_diff + w10_features_diff + ['label'], 
    "w15_features": w15_features_no_diff + w15_features_diff + ['label'], 
    "w20_features": w20_features_no_diff + w20_features_diff + ['label'], 
    "no_diff_features": w5_features_no_diff + w10_features_no_diff + w15_features_no_diff + w20_features_no_diff + ['label'], 
    "diff_features": w5_features_diff + w10_features_diff + w15_features_diff + w20_features_diff + ['label']
}

#### Data import

In [10]:
df = pd.read_csv("data/train/training_" + str(margin) + ".csv", index_col=[0])

In [11]:
print(len(df), "rows in the dataset")
print(df.columns)
df.describe()

33330 rows in the dataset
Index(['Gz', 'Ax', 'Ay', 'Gz_diff', 'Ax_diff', 'Ay_diff', 'Gz_mean_w5',
       'Ax_mean_w5', 'Ay_mean_w5', 'Gz_std_w5',
       ...
       'differencing_Ay_std_w20', 'differencing_Gz_min_w20',
       'differencing_Ax_min_w20', 'differencing_Ay_min_w20',
       'differencing_Gz_max_w20', 'differencing_Ax_max_w20',
       'differencing_Ay_max_w20', 'orient_discr', 'POSy_discr', 'label'],
      dtype='object', length=114)


,Gz,Ax,Ay,Gz_diff,Ax_diff,Ay_diff,Gz_mean_w5,Ax_mean_w5,Ay_mean_w5,Gz_std_w5,...,differencing_Ay_std_w20,differencing_Gz_min_w20,differencing_Ax_min_w20,differencing_Ay_min_w20,differencing_Gz_max_w20,differencing_Ax_max_w20,differencing_Ay_max_w20,orient_discr,POSy_discr,label
count,33330.0000,33330.0000,33330.0000,33330.0000,33330.0000,33330.0000,33330.0000,33330.0000,33330.0000,33330.0000,...,33330.0000,33330.0000,33330.0000,33330.0000,33330.0000,33330.0000,33330.0000,33330.0000,33330.0000,33330.0000
mean,-0.0000,-0.0000,0.0000,-0.0000,-0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000,...,0.0000,0.0000,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0071,25.0268,693.7214
std,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,...,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.9003,3.6446,486.4557
min,-3.5910,-9.4163,-6.6125,-8.9902,-8.4056,-6.2725,-3.1130,-8.1356,-4.8702,-0.9338,...,-4.0650,-3.9135,-11.6564,-7.0140,-5.2250,-3.2843,-3.8547,-1.0000,20.1000,0.0000
25%,-0.5174,-0.3436,-0.5260,-0.2134,-0.4330,-0.4971,-0.5244,-0.3965,-0.6088,-0.6870,...,-0.5792,-0.4764,-0.4117,-0.5304,-0.3900,-0.6150,-0.6252,-1.0000,21.1000,297.0000
50%,-0.2785,0.0458,-0.0226,0.0175,0.0045,0.0034,-0.2717,0.0523,-0.0462,-0.4343,...,-0.0641,0.0808,0.0600,0.0726,0.0148,-0.1864,-0.1131,0.0000,25.6000,603.5000
75%,0.6446,0.5491,0.5568,0.2488,0.4305,0.4885,0.6691,0.6927,0.5950,0.3711,...,0.5184,0.3879,0.4545,0.6099,0.5372,0.4121,0.5416,1.0000,28.9000,1003.0000
max,2.6170,4.2651,6.0183,9.0084,8.7169,7.9005,2.0682,2.3163,4.9736,7.5768,...,5.6213,4.0389,4.2209,3.8696,3.9312,5.3200,6.0086,1.0000,29.2000,2079.0000


# Machine learning: training phase

## Dataset preprocessing for machine learning models

In this section, RUL labels are converted to binary labels (`0/1`, namely `not_fault/fault`) in order to perform classification instead of regression.

For the `AutoEncoder` model, the dataset is partitioned such that the training set does not contain faults or samples which anticipate a fault. In other words, each sample must be compliant with the `good_samples_thr` threshold.

We basically need an entire section of dataset where faults are not present.

In [12]:
def build_dataset_for_ml_model(df, training_columns, split_size=0.75, as_list=False, ae=False):
    dfs = []
    df_main = df[training_columns]
    fault_indexes = df_main.index[df_main["label"] == 0].tolist() # list of indexes representing faults
    good_samples_thr = margin * 2
    
    previous = 0
    for fi in fault_indexes:
        dfs.append(df_main.iloc[previous:fi+1, :])
        previous = fi + 1
    
    rnd_list = list(range(len(dfs)))
    
    # If split_size is 1, there will be no val/test set
    train_size = floor(len(dfs) * split_size)
    train_index = rnd_list[:train_size]
    test_index = rnd_list[train_size:]
    train_rul = []
    test_rul = []
    
    if not as_list:
        first = True
        for ti in train_index:
            if not ae:
                to_concat = dfs[ti].copy()
            else:
                to_concat = dfs[ti][dfs[ti]["label"] >= good_samples_thr].copy()
            if first:
                training_set = to_concat
                first = False
            else:
                training_set = pd.concat([training_set, to_concat])

        first = True
        for ti in test_index:
            to_concat = dfs[ti].copy()
            if first:
                test_set = to_concat
                first = False
            else:
                test_set = pd.concat([test_set, to_concat])
        
        train_rul = training_set['label'].tolist()
        if split_size < 1:
            test_rul = test_set['label'].tolist()
        
        training_set['label'] = (training_set['label'] >= margin).map({True: 1, False: 0})
        if split_size < 1:
            test_set['label'] = (test_set['label'] >= margin).map({True: 1, False: 0})

        training_set = training_set.to_numpy()
        if split_size < 1:
            test_set = test_set.to_numpy()
        
    else:
        first = True
        for ti in train_index:
            if not ae:
                to_concat = dfs[ti].copy()
            else:
                to_concat = dfs[ti][dfs[ti]["label"] >= good_samples_thr].copy()
            if first:
                training_set = [to_concat]
                first = False
            else:
                training_set.append(to_concat)
                
        first = True
        for ti in test_index:
            to_concat = dfs[ti].copy()
            if first:
                test_set = [to_concat]
                first = False
            else:
                test_set.append(to_concat)
        
        for t in training_set:
            train_rul = train_rul + t['label'].tolist()
            t['label'] = (t['label'] >= margin).map({True: 1, False: 0})
        if split_size < 1:
            for t in test_set:
                test_rul = test_rul + t['label'].tolist()
                t['label'] = (t['label'] >= margin).map({True: 1, False: 0})
    if split_size < 1:
        return training_set, test_set
    return training_set

## Cost model for threshold optimization and performance evaluation

In [13]:
all_perf = []

In [14]:
BASE_FP = 0.2
BASE_FN = 1

def false_positive_cost(i, is_fault, fault_found):
    return BASE_FP

def false_negative_cost(i, is_fault, fault_found):
    if not fault_found:
        for j in range(1, margin + 1):
            if i + j < is_fault.shape[0] and not is_fault[i + j] or i + j >= is_fault.shape[0]:
                return (margin + 1 - j) * BASE_FN
    else:
        return 0

In [15]:
def threshold_optimization(signal, rul, start, end, n_steps):
    best_cost = sys.maxsize
    best_thr = -1
    all_cost = []
    all_thr = []
    is_fault = (rul == 0)
    
    for thr in np.linspace(start, end, n_steps):
        tmp_cost = 0
        fault_found = False
        for i in range(signal.shape[0]):
            if is_fault[i] and signal[i] >= thr:
                fault_found = True
            if not is_fault[i]:
                fault_found = False
            if not is_fault[i] and signal[i] >= thr:
                tmp_cost += false_positive_cost(i, is_fault, fault_found)
            elif is_fault[i] and signal[i] <= thr:
                tmp_cost += false_negative_cost(i, is_fault, fault_found)
        if tmp_cost < best_cost:
            best_thr = thr
            best_cost = tmp_cost
        all_cost.append(tmp_cost)
        all_thr.append(thr)

    return best_cost, best_thr, all_cost, all_thr

In [16]:
def plot_threshold(signal, thr, rul):

    plt.plot(signal, alpha=0.5)
    plt.plot(range(len(signal)), [thr] * len(signal))

    ranges = []
    signal_values = []
    for i in range(len(rul)):
        if rul[i] == 0:
            ranges.append(i)
            signal_values.append(signal[i])

    plt.scatter(ranges, signal_values, color="red", s=10)
    
    plt.ylabel('Alarm signal intensity')
    plt.xlabel('Time')
    plt.legend(['Alarm signal', "Threshold", 'Anomalies'], loc='upper right')
    plt.show()
    plt.show()

In [17]:
def performance_evaluation(signal, thr, rul):
    fp, fn, tp, tot_p = 0, 0, 0, 0
    cost = 0
    alarm = (signal >= thr)
    anticipation = []
    is_fault = (rul == 0)
    
    fault_found = False
    for i in range(len(rul)):
        if i > 0 and is_fault[i] and not is_fault[i - 1]:
            tot_p += 1
            start = i
        if is_fault[i] and not fault_found and alarm[i]:
            tp += 1
            fault_found = True
            anticipation.append((margin - 1) - (i - start))
        if (i < len(rul) - 1 and is_fault[i] and not is_fault[i + 1] and not fault_found) or (i == len(rul) - 1 and not fault_found):
            fn += 1 
        if is_fault[i] and signal[i] <= thr:
            cost += false_negative_cost(i, is_fault, fault_found)
        if not is_fault[i]:
            fault_found = False
            if alarm[i]:
                fp += 1
                cost += false_positive_cost(i, is_fault, fault_found)
        
    tot_a = sum(anticipation) / 10
    if sum(anticipation) > 0:
        mean_a = mean(anticipation) / 10
    else:
        mean_a = 0
    
    return [cost, mean_a, tp, fn, fp]

## Baseline: raw signal pre-anomaly detection

In [18]:
features_raw = [
 'Ax',
 'Ax_diff',
 'Ax_mean_w5',
 'Ax_std_w5',
 'Ax_min_w5',
 'Ax_max_w5',
 'Ax_mean_w10',
 'Ax_std_w10',
 'Ax_min_w10',
 'Ax_max_w10',
 'Ax_mean_w15',
 'Ax_std_w15',
 'Ax_min_w15',
 'Ax_max_w15',
 'Ax_mean_w20',
 'Ax_std_w20',
 'Ax_min_w20',
 'Ax_max_w20',
 'differencing_Ax',
 'differencing_Ax_diff',
 'differencing_Ax_mean_w5',
 'differencing_Ax_std_w5',
 'differencing_Ax_min_w5',
 'differencing_Ax_max_w5',
 'differencing_Ax_mean_w10',
 'differencing_Ax_std_w10',
 'differencing_Ax_min_w10',
 'differencing_Ax_max_w10',
 'differencing_Ax_mean_w15',
 'differencing_Ax_std_w15',
 'differencing_Ax_min_w15',
 'differencing_Ax_max_w15',
 'differencing_Ax_mean_w20',
 'differencing_Ax_std_w20',
 'differencing_Ax_min_w20',
 'differencing_Ax_max_w20',
]

for seed in seeds:
    for feature in features_raw:
        
        set_determinism(seed)
        
        training_columns = [feature, "label"]
        _, validation_set_raw = build_dataset_for_ml_model(df, training_columns=training_columns)
        
        val_raw_signal, val_raw_rul = -validation_set_raw[:, 0], validation_set_raw[:, -1]
        
        best_cost_raw, best_thr_raw, all_cost_raw, all_thr_raw = threshold_optimization(val_raw_signal, val_raw_rul, start=0, end=val_raw_signal.max(), n_steps=200)
        perf_raw = performance_evaluation(val_raw_signal, best_thr_raw, val_raw_rul)
        all_perf.append(["raw_signal", seed, feature, {}] + perf_raw)
        
        serialize_perf("raw_signal", seed=seed, columns_name=feature, training_columns=training_columns, 
                       params={}, params_idx=0, history=None, best_cost=best_cost_raw, best_thr=best_thr_raw, 
                       all_cost=all_cost_raw, all_thr=all_thr_raw, perf=perf_raw)
        
        # No model saving here: there is just the threshold

## Pre-anomaly detection with AutoEncoders

In [19]:
def build_autoencoder(input_size, hidden):
    input_shape = (input_size, )
    ae_x = keras.Input(shape=input_shape, dtype='float32')
    x = ae_x
    for h in hidden:
        x = layers.Dense(h, activation='relu')(x)
    ae_y = layers.Dense(input_size, activation='linear')(x)
    ae = keras.Model(ae_x, ae_y)   
    
    return ae

In [20]:
params_ae = [{"hidden_ae": [16, 8, 2, 8, 16]}, 
             {"hidden_ae": [64, 24, 9, 24, 64]}, 
             {"hidden_ae": [128, 56, 18, 56, 128]}]

In [21]:
for seed in seeds:
    for columns in features:
        for params_idx, params in enumerate(params_ae):

            set_determinism(seed)

            training_set_ae, validation_set_ae = build_dataset_for_ml_model(df, training_columns=features[columns], ae=True)

            train_cols_ae = training_set_ae.shape[1] - 1
            ae = build_autoencoder(input_size=train_cols_ae, hidden=params["hidden_ae"])
            ae.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
                       loss='mse')
            cb_ae = [callbacks.EarlyStopping(patience=patience, restore_best_weights=True)]
            history_ae = ae.fit(training_set_ae[:, :-1], training_set_ae[:, :-1], validation_split=validation_split,
                                callbacks=cb_ae, batch_size=batch_size, epochs=epochs, verbose=0)
            
            preds_ae = ae.predict(validation_set_ae[:, :-1])
            
            signal_ae = pd.Series(data=np.sum(np.square(preds_ae - validation_set_ae[:, :-1]), axis=1))
            rul_ae = validation_set_ae[:, -1]

            best_cost_ae, best_thr_ae, all_cost_ae, all_thr_ae = threshold_optimization(signal_ae, rul_ae, start=0, end=signal_ae.max(), n_steps=200)
            
            perf_ae = performance_evaluation(signal_ae, best_thr_ae, rul_ae)
            all_perf.append(["autoencoder", seed, columns, params] + perf_ae)
            
            serialize_perf("autoencoder", seed=seed, columns_name=columns, training_columns=features[columns], 
                           params=params, params_idx=params_idx, history=history_ae, best_cost=best_cost_ae, best_thr=best_thr_ae, 
                           all_cost=all_cost_ae, all_thr=all_thr_ae, perf=perf_ae)
            
            ae.save(models_path + "autoencoder" + "-" + columns + "-s" + str(seed) + "-p" + str(params_idx))

221/221 [==============================] - 0s 566us/step
INFO:tensorflow:Assets written to: ram://31e490ef-fa26-48f4-8acf-57e7bfdcf9de/assets
INFO:tensorflow:Assets written to: models/autoencoder-all_features-s100-p0/assets
221/221 [==============================] - 0s 559us/step
INFO:tensorflow:Assets written to: ram://3c09deab-20f5-493e-91d3-3cac7d05887f/assets
INFO:tensorflow:Assets written to: models/autoencoder-all_features-s100-p1/assets
221/221 [==============================] - 0s 613us/step
INFO:tensorflow:Assets written to: ram://73b9fdbe-6f42-4751-9c48-e59c2d8f330c/assets
INFO:tensorflow:Assets written to: models/autoencoder-all_features-s100-p2/assets
221/221 [==============================] - 0s 580us/step
INFO:tensorflow:Assets written to: ram://fabcc5b2-2721-4317-8746-37882e428242/assets
INFO:tensorflow:Assets written to: models/autoencoder-w5_features-s100-p0/assets
221/221 [==============================] - 0s 577us/step
INFO:tensorflow:Assets written to: ram://a3b3f52

INFO:tensorflow:Assets written to: models/autoencoder-no_diff_features-s200-p0/assets
221/221 [==============================] - 0s 519us/step
INFO:tensorflow:Assets written to: ram://6eb147be-da24-4d35-8017-4b9f9689fc80/assets
INFO:tensorflow:Assets written to: models/autoencoder-no_diff_features-s200-p1/assets
221/221 [==============================] - 0s 545us/step
INFO:tensorflow:Assets written to: ram://4d56810a-6d5f-49ca-9db7-50de6ba9af3a/assets
INFO:tensorflow:Assets written to: models/autoencoder-no_diff_features-s200-p2/assets
221/221 [==============================] - 0s 568us/step
INFO:tensorflow:Assets written to: ram://6c00a5a6-8d5c-4e6c-8ce7-017fb6aa0aee/assets
INFO:tensorflow:Assets written to: models/autoencoder-diff_features-s200-p0/assets
221/221 [==============================] - 0s 591us/step
INFO:tensorflow:Assets written to: ram://03ae1f61-edac-4766-86d1-c1381b2210aa/assets
INFO:tensorflow:Assets written to: models/autoencoder-diff_features-s200-p1/assets
221/221 

## RUL estimation with Dense Neural Networks

In [22]:
def build_classifier(input_size, hidden):
    input_shape = (input_size,)
    model_in = keras.Input(shape=input_shape, dtype='float32')
    x = model_in
    for h in hidden:
        x = layers.Dense(h, activation='relu')(x)
    model_out = layers.Dense(1, activation='sigmoid')(x)
    model = keras.Model(model_in, model_out)
    return model

In [23]:
params_mlp = [{"hidden_mlp": []},
              {"hidden_mlp": [32]},
              {"hidden_mlp": [64, 32]}, 
              {"hidden_mlp": [128, 64, 32]}]

**Class weights** are useful when you have an **unbalanced dataset** and you want to improve single-label classification results. With class weights, you can weight more the samples belonging to the rarest class.

In [24]:
for seed in seeds:
    for columns in features:
        for params_idx, params in enumerate(params_mlp):

            set_determinism(seed)
            
            training_set_mlp, validation_set_mlp = build_dataset_for_ml_model(df, training_columns=features[columns])
            counts_mlp = pd.Series(training_set_mlp[:, -1]).value_counts(normalize=True)
            class_weight_mlp = {0: 1/counts_mlp[0], 1: 1/counts_mlp[1]}
            
            input_size_mlp = training_set_mlp.shape[1] - 1
            mlp = build_classifier(input_size=input_size_mlp, hidden=params["hidden_mlp"])
            mlp.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss='binary_crossentropy')
            cb_mlp = [callbacks.EarlyStopping(patience=patience, restore_best_weights=True)]
            history_mlp = mlp.fit(training_set_mlp[:, :-1], training_set_mlp[:, -1], validation_split=validation_split,
                                  callbacks=cb_mlp, class_weight=class_weight_mlp,
                                  batch_size=batch_size, epochs=epochs, verbose=0)
            
            preds_mlp = mlp.predict(validation_set_mlp[:, :-1]).ravel()
            
            signal_mlp = pd.Series(data=(1 - preds_mlp))
            rul_mlp = validation_set_mlp[:, -1]

            best_cost_mlp, best_thr_mlp, all_cost_mlp, all_thr_mlp = threshold_optimization(signal_mlp, rul_mlp, start=0, end=signal_mlp.max(), n_steps=200)
            
            perf_mlp = performance_evaluation(signal_mlp, best_thr_mlp, rul_mlp)
            all_perf.append(["mlp", seed, columns, params] + perf_mlp)
            
            serialize_perf("mlp", seed=seed, columns_name=columns, training_columns=features[columns], 
                           params=params, params_idx=params_idx, history=history_mlp, best_cost=best_cost_mlp, best_thr=best_thr_mlp, 
                           all_cost=all_cost_mlp, all_thr=all_thr_mlp, perf=perf_mlp)
            
            mlp.save(models_path + "mlp" + "-" + columns + "-s" + str(seed) + "-p" + str(params_idx))

221/221 [==============================] - 0s 462us/step
INFO:tensorflow:Assets written to: ram://7c1f2538-f9a2-45a2-af11-008956144e35/assets
INFO:tensorflow:Assets written to: models/mlp-all_features-s100-p0/assets
221/221 [==============================] - 0s 524us/step
INFO:tensorflow:Assets written to: ram://bf39976c-0d59-48cf-b77b-6d490e120f06/assets
INFO:tensorflow:Assets written to: models/mlp-all_features-s100-p1/assets
221/221 [==============================] - 0s 481us/step
INFO:tensorflow:Assets written to: ram://d30d593b-a219-4e76-a6b4-f901e7589299/assets
INFO:tensorflow:Assets written to: models/mlp-all_features-s100-p2/assets
221/221 [==============================] - 0s 527us/step
INFO:tensorflow:Assets written to: ram://163a17ae-e81e-433c-97fc-1d1bf883960b/assets
INFO:tensorflow:Assets written to: models/mlp-all_features-s100-p3/assets
221/221 [==============================] - 0s 464us/step
INFO:tensorflow:Assets written to: ram://41875fec-05d8-4de9-88d2-1d7c054485b7/a

221/221 [==============================] - 0s 529us/step
INFO:tensorflow:Assets written to: ram://27a99f1e-2471-4440-9a61-efa76ed88050/assets
INFO:tensorflow:Assets written to: models/mlp-w10_features-s200-p2/assets
221/221 [==============================] - 0s 537us/step
INFO:tensorflow:Assets written to: ram://92498615-ecd4-4d19-9ebc-83631c87dee3/assets
INFO:tensorflow:Assets written to: models/mlp-w10_features-s200-p3/assets
221/221 [==============================] - 0s 447us/step
INFO:tensorflow:Assets written to: ram://098f097a-ef4a-4e4b-82e6-b438259293fd/assets
INFO:tensorflow:Assets written to: models/mlp-w15_features-s200-p0/assets
221/221 [==============================] - 0s 507us/step
INFO:tensorflow:Assets written to: ram://fdf738a2-ea76-4c44-980b-88bd8518995e/assets
INFO:tensorflow:Assets written to: models/mlp-w15_features-s200-p1/assets
221/221 [==============================] - 0s 506us/step
INFO:tensorflow:Assets written to: ram://3f8f8392-5627-47fc-aefb-4e584af41e46/a

221/221 [==============================] - 0s 479us/step
INFO:tensorflow:Assets written to: ram://eacb48b0-8783-4d55-a6c4-a85350d8fc78/assets
INFO:tensorflow:Assets written to: models/mlp-no_diff_features-s300-p0/assets
221/221 [==============================] - 0s 474us/step
INFO:tensorflow:Assets written to: ram://ee2d1979-f8fb-4af4-b172-4474414e7e98/assets
INFO:tensorflow:Assets written to: models/mlp-no_diff_features-s300-p1/assets
221/221 [==============================] - 0s 501us/step
INFO:tensorflow:Assets written to: ram://54f8c9f8-7caf-4b95-9b9f-675b62bfa545/assets
INFO:tensorflow:Assets written to: models/mlp-no_diff_features-s300-p2/assets
221/221 [==============================] - 0s 527us/step
INFO:tensorflow:Assets written to: ram://94fc0fb9-d9a2-4ea4-9b4f-55cae1b2c17d/assets
INFO:tensorflow:Assets written to: models/mlp-no_diff_features-s300-p3/assets
221/221 [==============================] - 0s 448us/step
INFO:tensorflow:Assets written to: ram://a2371bb5-9547-4b4f-868

## RUL estimation with Convolutional Neural Networks

In [25]:
def sliding_window_2D(data, w_len, stride=1):
    # Get shifted tables
    m = len(data)
    lt = [data.iloc[i:m-w_len+i+1:stride, :].values for i in range(w_len)]
    # Reshape to add a new axis
    s = lt[0].shape
    for i in range(w_len):
        lt[i] = lt[i].reshape(s[0], 1, s[1])
    # Concatenate
    wdata = np.concatenate(lt, axis=1)
    return wdata


def sliding_window_by_fault(data, cols, w_len, stride=1):
    l_w, l_r = [], []
    for gdata in data:
        # Apply a sliding window
        tmp_w = sliding_window_2D(gdata[cols], w_len, stride)
        # Build the RUL vector
        tmp_r = gdata['label'].iloc[w_len-1::stride]
        # Store everything
        l_w.append(tmp_w)
        l_r.append(tmp_r)
    res_w = np.concatenate(l_w)
    res_r = np.concatenate(l_r)
    return res_w, res_r

In [26]:
def build_cnn_regressor(input_size, filters, kernel_size, hidden, w_len):
    input_shape = (w_len, input_size)
    model_in = keras.Input(shape=input_shape, dtype='float32')
    model_out = layers.Conv1D(filters, kernel_size=kernel_size, 
                              activation='relu')(model_in)
    model_out = layers.Flatten()(model_out)
    for h in hidden:
        model_out = layers.Dense(h, activation='relu')(model_out)
    model_out = layers.Dense(1, activation='sigmoid')(model_out)
    model = keras.Model(model_in, model_out)
    return model

In [27]:
params_cnn = [{"filters": 1, "kernel_size": 3, "hidden": [32], "w_len": 5},
              {"filters": 4, "kernel_size": 3, "hidden": [32], "w_len": 5},
              {"filters": 1, "kernel_size": 5, "hidden": [32], "w_len": 5},
              {"filters": 4, "kernel_size": 5, "hidden": [32], "w_len": 5},
              {"filters": 4, "kernel_size": 5, "hidden": [64, 32], "w_len": 5},
              {"filters": 1, "kernel_size": 3, "hidden": [32], "w_len": 10},
              {"filters": 4, "kernel_size": 3, "hidden": [32], "w_len": 10},
              {"filters": 1, "kernel_size": 5, "hidden": [32], "w_len": 10},
              {"filters": 4, "kernel_size": 5, "hidden": [32], "w_len": 10},
              {"filters": 4, "kernel_size": 5, "hidden": [64, 32], "w_len": 10},
              {"filters": 4, "kernel_size": 7, "hidden": [128, 64, 32], "w_len": 10}]

In [28]:
for seed in seeds:
    for columns in features:
        for params_idx, params in enumerate(params_cnn):
            
            set_determinism(seed)
            
            traning_set_cnn, validation_set_cnn = build_dataset_for_ml_model(df, training_columns=features[columns], as_list=True)
            tr_sw, tr_sw_r = sliding_window_by_fault(traning_set_cnn, features[columns], params["w_len"])
            val_sw, val_sw_r = sliding_window_by_fault(validation_set_cnn, features[columns], params["w_len"])
            counts_cnn = pd.Series(tr_sw_r).value_counts(normalize=True)
            class_weight_cnn = {0: 1/counts_cnn[0], 1: 1/counts_cnn[1]}

            input_size_cnn = tr_sw[0].shape[1]
            cnn = build_cnn_regressor(input_size=input_size_cnn, filters=params["filters"],
                                      kernel_size=params["kernel_size"], hidden=params["hidden"], w_len=params["w_len"])
            cnn.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
                        loss='binary_crossentropy')
            cb_cnn = [callbacks.EarlyStopping(patience=patience, restore_best_weights=True)]
            history_cnn = cnn.fit(tr_sw, tr_sw_r, validation_split=validation_split,
                                  callbacks=cb_cnn,
                                  class_weight=class_weight_cnn,
                                  batch_size=batch_size, epochs=epochs, verbose=0)
            
            preds_cnn = cnn.predict(val_sw).ravel()
            
            signal_cnn = pd.Series(data=(1 - preds_cnn))
            rul_cnn = val_sw_r

            best_cost_cnn, best_thr_cnn, all_cost_cnn, all_thr_cnn = threshold_optimization(signal_cnn, rul_cnn, start=0, end=signal_cnn.max(), n_steps=200)
            
            perf_cnn = performance_evaluation(signal_cnn, best_thr_cnn, rul_cnn)
            all_perf.append(["conv_nn", seed, columns, params] + perf_cnn)
            
            serialize_perf("conv_nn", seed=seed, columns_name=columns, training_columns=features[columns], 
                           params=params, params_idx=params_idx, history=history_cnn, best_cost=best_cost_cnn, best_thr=best_thr_cnn, 
                           all_cost=all_cost_cnn, all_thr=all_thr_cnn, perf=perf_cnn)
            
            cnn.save(models_path + "conv_nn" + "-" + columns + "-s" + str(seed) + "-p" + str(params_idx))

220/220 [==============================] - 0s 647us/step


INFO:tensorflow:Assets written to: ram://edd4b914-42a3-4160-a31f-344e4787f793/assets


INFO:tensorflow:Assets written to: ram://edd4b914-42a3-4160-a31f-344e4787f793/assets


INFO:tensorflow:Assets written to: models/conv_nn-all_features-s100-p0/assets


INFO:tensorflow:Assets written to: models/conv_nn-all_features-s100-p0/assets


...

219/219 [==============================] - 0s 653us/step


INFO:tensorflow:Assets written to: ram://5062f2fd-19a4-4ada-aa66-68edbba63cec/assets


INFO:tensorflow:Assets written to: ram://5062f2fd-19a4-4ada-aa66-68edbba63cec/assets


INFO:tensorflow:Assets written to: models/conv_nn-diff_features-s300-p10/assets


INFO:tensorflow:Assets written to: models/conv_nn-diff_features-s300-p10/assets


## RUL estimation with Recurrent Neural Networks (LSTM and GRU)

In [29]:
def create_dataset_3D(X, y, time_steps):
    Xs, ys = [], []
    for i in range(len(X)-time_steps):
        v = X[i:i+time_steps, :]
        Xs.append(v)
        ys.append(y[i+time_steps])
    return np.array(Xs), np.array(ys).reshape(-1, 1)

In [30]:
def create_model_cl(units, m, X_train, lr):
    model = keras.Sequential()
    model.add(m(units=units, return_sequences=True,
                input_shape=[X_train.shape[1], X_train.shape[2]]))
    model.add(m(units=units))
    model.add(keras.layers.Dense(units=1, activation="sigmoid"))
    model.compile(loss='binary_crossentropy', 
                  optimizer=keras.optimizers.Adam(learning_rate=lr))
    return model

In [31]:
params_rnn = [{"time_steps": 5, "units": 64},
              {"time_steps": 5, "units": 128},
              {"time_steps": 10, "units": 64},
              {"time_steps": 10, "units": 128}]

In [32]:
for seed in seeds:
    for columns in features:
        for params_idx, params in enumerate(params_rnn):
            
            set_determinism(seed)

            traning_set_rnn, validation_set_rnn = build_dataset_for_ml_model(df, training_columns=features[columns])
            X_train_rnn, y_train_rnn = create_dataset_3D(traning_set_rnn[:, :-1], 
                                                         traning_set_rnn[:, -1], 
                                                         params["time_steps"])
            X_val_rnn, y_val_rnn = create_dataset_3D(validation_set_rnn[:, :-1], 
                                                     validation_set_rnn[:, -1],   
                                                     params["time_steps"])
            counts_rnn = pd.Series(tr_sw_r).value_counts(normalize=True)
            class_weight_rnn = {0: 1/counts_rnn[0], 1: 1/counts_rnn[1]}
            
            model_gru = create_model_cl(params["units"], keras.layers.GRU, X_train_rnn, learning_rate)
            early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',
                                                       patience=patience)
            history_gru = model_gru.fit(X_train_rnn, y_train_rnn, epochs=epochs,
                                        class_weight=class_weight_rnn,
                                        validation_split=validation_split, batch_size=batch_size,
                                        shuffle=False, callbacks=[early_stop])
            
            model_lstm = create_model_cl(params["units"], keras.layers.LSTM, X_train_rnn, learning_rate)
            early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',
                                                       patience=patience)
            history_lstm = model_lstm.fit(X_train_rnn, y_train_rnn, epochs=epochs,
                                          class_weight=class_weight_rnn,
                                          validation_split=validation_split, batch_size=batch_size,
                                          shuffle=False, callbacks=[early_stop], verbose=0)
            
            preds_lstm = model_lstm.predict(X_val_rnn).ravel()
            preds_gru = model_gru.predict(X_val_rnn).ravel()
            
            signal_lstm = pd.Series(data=(1 - preds_lstm))
            rul_lstm = y_val_rnn
            signal_gru = pd.Series(data=(1 - preds_gru))
            rul_gru = y_val_rnn
            
            best_cost_lstm, best_thr_lstm, all_cost_lstm, all_thr_lstm = threshold_optimization(signal_lstm, rul_lstm, start=0, end=signal_lstm.max(), n_steps=200)
            best_cost_gru, best_thr_gru, all_cost_gru, all_thr_gru = threshold_optimization(signal_gru, rul_gru, start=0, end=signal_gru.max(), n_steps=200)
            
            perf_lstm = performance_evaluation(signal_lstm, best_thr_lstm, rul_lstm)
            perf_gru = performance_evaluation(signal_gru, best_thr_gru, rul_gru)
            all_perf.append(["lstm", seed, columns, params] + perf_lstm)  
            all_perf.append(["gru", seed, columns, params] + perf_gru)
            
            serialize_perf("lstm", seed=seed, columns_name=columns, training_columns=features[columns], 
                           params=params, params_idx=params_idx, history=history_lstm, best_cost=best_cost_lstm, best_thr=best_thr_lstm, 
                           all_cost=all_cost_lstm, all_thr=all_thr_lstm, perf=perf_lstm)
            
            model_lstm.save(models_path + "lstm" + "-" + columns + "-s" + str(seed) + "-p" + str(params_idx))
            
            serialize_perf("gru", seed=seed, columns_name=columns, training_columns=features[columns], 
                           params=params, params_idx=params_idx, history=history_gru, best_cost=best_cost_gru, best_thr=best_thr_gru, 
                           all_cost=all_cost_gru, all_thr=all_thr_gru, perf=perf_gru)
            
            model_gru.save(models_path + "gru" + "-" + columns + "-s" + str(seed) + "-p" + str(params_idx))

Epoch 1/3
657/657 [==============================] - 4s 3ms/step - loss: 0.9000 - val_loss: 0.7056
Epoch 2/3
657/657 [==============================] - 1s 2ms/step - loss: 0.6320 - val_loss: 0.5254
Epoch 3/3
221/221 [==============================] - 1s 931us/step


INFO:tensorflow:Assets written to: ram://7377962c-17aa-4e06-b68f-f3fbbad5a4e5/assets


INFO:tensorflow:Assets written to: ram://7377962c-17aa-4e06-b68f-f3fbbad5a4e5/assets


INFO:tensorflow:Assets written to: models/lstm-all_features-s100-p0/assets


INFO:tensorflow:Assets written to: models/lstm-all_features-s100-p0/assets


...

INFO:tensorflow:Assets written to: models/lstm-diff_features-s300-p3/assets


INFO:tensorflow:Assets written to: ram://1aa2eb7b-8273-422b-b6a6-34931e771a04/assets


INFO:tensorflow:Assets written to: ram://1aa2eb7b-8273-422b-b6a6-34931e771a04/assets


INFO:tensorflow:Assets written to: models/gru-diff_features-s300-p3/assets


INFO:tensorflow:Assets written to: models/gru-diff_features-s300-p3/assets


## RUL estimation with Recurrent Neural Networks (BiLSTM)

In [33]:
def create_model_bilstm_cl(units, X_train, lr):
    model = keras.Sequential()
    model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=units,                             
              return_sequences=True),
              input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=units)))
    model.add(keras.layers.Dense(1, activation="sigmoid"))
    model.compile(loss='binary_crossentropy', 
                  optimizer=keras.optimizers.Adam(learning_rate=lr))
    return model

In [34]:
params_bilstm = [{"time_steps": 5, "units": 64},
                 {"time_steps": 5, "units": 128},
                 {"time_steps": 10, "units": 64},
                 {"time_steps": 10, "units": 128}]

In [35]:
for seed in seeds:
    for columns in features:
        for params_idx, params in enumerate(params_bilstm):
            
            set_determinism(seed)

            traning_set_bilstm, validation_set_bilstm = build_dataset_for_ml_model(df, training_columns=features[columns])
            X_train_bilstm, y_train_bilstm = create_dataset_3D(traning_set_bilstm[:, :-1], 
                                                               traning_set_bilstm[:, -1], 
                                                               params["time_steps"])
            X_val_bilstm, y_val_bilstm = create_dataset_3D(validation_set_bilstm[:, :-1], 
                                                           validation_set_bilstm[:, -1],   
                                                           params["time_steps"])
            counts_bilstm = pd.Series(tr_sw_r).value_counts(normalize=True)
            class_weight_bilstm = {0: 1/counts_bilstm[0], 1: 1/counts_bilstm[1]}
            model_bilstm = create_model_bilstm_cl(params["units"], X_train_bilstm, learning_rate)
            early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',
                                                       patience=patience)
            history_bilstm = model_bilstm.fit(X_train_bilstm, y_train_bilstm, epochs=epochs,
                                              class_weight=class_weight_bilstm,
                                              validation_split=validation_split, batch_size=batch_size,
                                              shuffle=False, callbacks=[early_stop], verbose=0)
            
            preds_bilstm = model_bilstm.predict(X_val_bilstm).ravel()
            
            signal_bilstm = pd.Series(data=(1 - preds_bilstm))
            rul_bilstm = y_val_bilstm

            best_cost_bilstm, best_thr_bilstm, all_cost_bilstm, all_thr_bilstm = threshold_optimization(signal_bilstm, rul_bilstm, start=0, end=signal_bilstm.max(), n_steps=200)
            
            perf_bilstm = performance_evaluation(signal_bilstm, best_thr_bilstm, rul_bilstm)
            all_perf.append(["bilstm", seed, columns, params] + perf_bilstm)
            
            serialize_perf("bilstm", seed=seed, columns_name=columns, training_columns=features[columns], 
                           params=params, params_idx=params_idx, history=history_bilstm, best_cost=best_cost_bilstm, best_thr=best_thr_bilstm, 
                           all_cost=all_cost_bilstm, all_thr=all_thr_bilstm, perf=perf_bilstm)
            
            model_bilstm.save(models_path + "bilstm" + "-" + columns + "-s" + str(seed) + "-p" + str(params_idx))

221/221 [==============================] - 1s 1ms/step


INFO:tensorflow:Assets written to: ram://f7d33c2e-8cfe-485e-80cc-8af2a3433b1a/assets


INFO:tensorflow:Assets written to: ram://f7d33c2e-8cfe-485e-80cc-8af2a3433b1a/assets


INFO:tensorflow:Assets written to: models/bilstm-all_features-s100-p0/assets


INFO:tensorflow:Assets written to: models/bilstm-all_features-s100-p0/assets


...

221/221 [==============================] - 1s 2ms/step


INFO:tensorflow:Assets written to: ram://06814328-5ade-4e47-8224-d6616919bcc7/assets


INFO:tensorflow:Assets written to: ram://06814328-5ade-4e47-8224-d6616919bcc7/assets


INFO:tensorflow:Assets written to: models/bilstm-diff_features-s300-p3/assets


INFO:tensorflow:Assets written to: models/bilstm-diff_features-s300-p3/assets


## Analysis over the validation set

In [36]:
df_res = pd.DataFrame(all_perf, columns=["model", "seed", "columns", "params", "cost", "anticipation", "detected_faults", "missed_faults", "false_alarms"])
df_res.to_csv("training_summary_" + str(margin) + ".csv")
df_res

,model,seed,columns,params,cost,anticipation,detected_faults,missed_faults,false_alarms
0,raw_signal,100,Ax,{},32.0000,0.2167,6,1,0
1,raw_signal,100,Ax_diff,{},29.8000,0.3167,6,1,34
2,raw_signal,100,Ax_mean_w5,{},58.4000,0.0833,6,1,12
3,raw_signal,100,Ax_std_w5,{},105.2000,0.0000,0,7,1
4,raw_signal,100,Ax_min_w5,{},32.0000,0.2167,6,1,0
...,...,...,...,...,...,...,...,...,...
733,bilstm,300,no_diff_features,"{'time_steps': 10, 'units': 128}",40.4000,0.2500,6,1,47
734,bilstm,300,diff_features,"{'time_steps': 5, 'units': 64}",54.0000,0.1333,6,1,35
735,bilstm,300,diff_features,"{'time_steps': 5, 'units': 128}",50.0000,0.1667,6,1,40
736,bilstm,300,diff_features,"{'time_steps': 10, 'units': 64}",59.0000,0.1000,6,1,20
